<a href="https://colab.research.google.com/github/joshsomma/Complete-Python-Bootcamp/blob/master/Football_Pier_Data_Analytics_NEXT_UK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
<img src="https://storage.googleapis.com/football-pier-colab/footballpier-banner.jpg" width="1024">
<hr>
<h1> Football Pier - Data Analytics with BigQuery</h1>
<hr>
<img src="https://storage.googleapis.com/football-pier-colab/123ABC.jpg" width="240">
</div>

In [0]:
#@title Enter your `PLAYER ID` below
PLAYER_ID = '48XGTZ' #@param{type:"string"}

## Set up the environment

In [0]:
#@title Import Python libraries
# Basic Python data science libraries
import pandas as pd
import numpy as np

# Package for interfacing w/ BigQuery from Python
from google.cloud import bigquery

# Imports for plotly in Colab
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline

from google.colab import data_table


The authentication step in the next cell will require manually going through some pop-up screens and copy/pasting an authentication code from another window back into the cell to complete (on the 1st run; may run automatically thereafter).

In [0]:
#@title Provide Google Credentials to Colab Runtime
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### Set up the BigQuery Python client

If you have never used Google Cloud before, you will first need to activate your account and create a project. You can follow the [quick start instructions here](https://cloud.google.com/bigquery/docs/quickstarts/quickstart-web-ui).

Complete up until `Enable the API`, then proceed below:

In [0]:
#@title Enter GCP/BigQuery Project ID (replace with your PROJECT_ID below)
PROJECT_ID = 'careful-triumph-259607' #@param{type:"string"}
# 'careful-triumph-259607'
# 'gcp-data-science-demo'

# Create BigQuery client with given project
bq_client = bigquery.Client(project = PROJECT_ID)

### Other Setup

In [0]:
#@title Other Setup
#Various image file Links
goal_mouth_image_file = "https://raw.githubusercontent.com/alokpattani/soccer_penalty_kicks/master/Goal_Mouth_Image_Tight_NoColor.png"

white_ball_image_file = "https://raw.githubusercontent.com/alokpattani/soccer_penalty_kicks/master/Futbol_White.png"

green_ball_image_file = "https://raw.githubusercontent.com/alokpattani/soccer_penalty_kicks/master/Futbol_Green.png"

red_x_image_file = (
  "https://raw.githubusercontent.com/alokpattani/soccer_penalty_kicks/master/Red_X.png")

pk_score_prob_heat_map_image_file = (
  "https://raw.githubusercontent.com/alokpattani/soccer_penalty_kicks/master/Penalty%20Kicks%20Score%20Probability%20Heat%20Map%20Narrow.png")

# Function to return pandas df from BigQuery SQL query
def get_df_from_query(sqlquery, client = bq_client):
  '''
  Given str query `sqlquery`,
  returns results from BigQuery as a pandas DataFrame
  '''
  df = client.query(sqlquery).result().to_dataframe()
  return df

#Visualize Ball at Goal Data for Your Kicks, All Kicks at the Event, and Professional's Kicks!
In this section, we'll look the position of the ball __as it crosses the goal mouth__ for:


*   your particular kicks
*   all kicks at the event
*   "real" penalty kicks taken by professional players in real matches (from a different data set).

Credits:
*   [Plotly](https://plot.ly/) for enabling great interactive plotting in Colab
*   [TruMedia Networks](http://www.trumedianetworks.com/) for the goal image and plotting penalty kicks inspiration
*   [Font Awesome](https://fontawesome.com/v4.7.0/icon/futbol-o) for the ball image
*   [Opta](https://www.optasports.com/) for penalty kick data from various professional competitions






## Compare Your Kicks to All at This Event

For each kick at this event, we read in location data for the ball at the time it crosses the goal mouth (`impact = 1` in the table) and calculate distance to the nearest target, from BigQuery. We display the data in an interactive table, with your kicks starting as the top rows(denoted by `is_you = TRUE` in the table).

In [0]:
#@title Read in Data on Ball at Goal for All Kicks at This Event from BigQuery
ball_at_goal_coordinates = get_df_from_query('''
  # Use select distinct to try to avoid duplicate rows for same kick
  SELECT DISTINCT 
    player, kickno, camera, ts_ball, impact,

    ## Modify target (x, y) to get left/right targets appropriately
    LEAST(target1_x, target2_x) AS left_target_x, 
    IF(target1_x = LEAST(target1_x, target2_x), target1_y, target2_y)
      AS left_target_y,
    GREATEST(target1_x, target2_x) AS right_target_x, 
    IF(target1_x = GREATEST(target1_x, target2_x), target1_y, target2_y)
      AS right_target_y,

    ball_center_x, ball_center_y, 
    ball_box_x1, ball_box_y1, ball_box_x2, ball_box_y2,
    
    LEAST(
      SQRT(POW(ball_center_x - target1_x, 2) + POW(ball_center_y - target1_y, 2)),
      SQRT(POW(ball_center_x - target2_x, 2) + POW(ball_center_y - target2_y, 2))
      ) AS dist_to_nearest_target

  FROM 
    `football-pier.goal.ball`

  WHERE
    # Filter to only 1st 3 kicks for given player
    kickno <= 3 AND
    # Filter to get ball position only when it crosses goal mouth (impact = 1)
    impact = 1
  ''')

# Add field to mark your kicks
ball_at_goal_coordinates['is_you'] = (ball_at_goal_coordinates['player']
  == PLAYER_ID)

# Display interactive data table of ball at goal coordinates
data_table.DataTable(
  ball_at_goal_coordinates
  .sort_values(['is_you', 'player', 'kickno'], 
    ascending = [False, True, True])
  .set_index(['is_you'])
  .reset_index()
  .round(decimals = 3)
  )

,is_you,player,kickno,camera,ts_ball,impact,left_target_x,left_target_y,right_target_x,right_target_y,ball_center_x,ball_center_y,ball_box_x1,ball_box_y1,ball_box_x2,ball_box_y2,dist_to_nearest_target
0,True,48XGTZ,1,goal,3.498,1,0.321,0.377,0.683,0.371,0.663,0.428,0.652,0.407,0.675,0.448,0.060
1,True,48XGTZ,2,goal,2.798,1,0.321,0.377,0.683,0.371,0.666,0.487,0.653,0.462,0.679,0.511,0.117
2,True,48XGTZ,3,goal,3.248,1,0.321,0.377,0.684,0.371,0.720,0.224,0.712,0.210,0.728,0.238,0.151
3,False,22SGZG,1,goal,12.816,1,0.272,0.385,0.639,0.394,0.297,0.352,0.288,0.336,0.305,0.367,0.041
4,False,22SGZG,2,goal,2.274,1,0.273,0.385,0.639,0.395,0.279,0.322,0.271,0.308,0.287,0.337,0.063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,False,ZL2WKD,2,goal,3.823,1,0.293,0.429,0.658,0.437,0.386,0.485,0.379,0.471,0.394,0.499,0.108
634,False,ZL2WKD,3,goal,3.523,1,0.293,0.429,0.658,0.437,0.330,0.360,0.322,0.345,0.338,0.376,0.078
635,False,ZQ2LGK,1,goal,3.148,1,0.290,0.338,0.659,0.342,0.312,0.374,0.303,0.358,0.320,0.390,0.042
636,False,ZQ2LGK,2,goal,3.698,1,0.290,0.338,0.659,0.342,0.340,0.321,0.332,0.305,0.348,0.337,0.053


Below is a plot of the ball location (as it crossed the goal mouth) for all kicks attempted during this event. Your kicks are highlighted with the green, slightly larger soccer balls. Hover over any ball to see more info on that kick, including distance to nearest target. Use your mouse to zoom in and out of different sections, and see the menu in the top right of the output area for even more options.

***Goal mouth and target images are placed approximately (not exactly) on the plot below, for illustration purposes.*** Distance calculations are relative to the target coordinates read in *on each individual kick*, not the fixed targets shown in the plot below.

In [0]:
#@title Interactive Plot of All Kicks from This Event on Image of Goal Mouth
def GetShotChartWithBallsAtGoalMouth(
  ball_at_goal_coordinates_df, dicts_of_images_to_add = [],
  figure_x_min = 0, figure_x_max = 1, figure_y_min = 0, figure_y_max = 1,
  figure_height = 600, figure_width_to_height_ratio = (1920 / 1080),
  plot_hoverinfo = 'x+y+text', add_dist_to_text = True
  ):

  all_dicts_of_images_to_add = dicts_of_images_to_add

  default_ball_sizex = 0.01/400 * figure_height
  default_ball_sizey = figure_width_to_height_ratio * default_ball_sizex

  for index, row in ball_at_goal_coordinates_df.iterrows():
    # Add multiplier to make player's ball appear larger, other participants' balls smaller
    ball_size_multiplier = (1.15 if(row['is_you']) else 0.85)
    this_ball_sizex = default_ball_sizex * ball_size_multiplier
    this_ball_sizey = default_ball_sizey * ball_size_multiplier

    this_ball_plot_dict = dict(
      source = (green_ball_image_file if row['is_you'] else
        white_ball_image_file),
      xref = "x",
      yref = "y",
      x = row['ball_center_x'] - (this_ball_sizex / 2),
      y = row['ball_center_y'] + (this_ball_sizey / 2),
      xanchor = 'left',
      yanchor = 'bottom',
      sizex = this_ball_sizex,
      sizey = this_ball_sizey,
      sizing = "stretch",
      opacity = (1 if row['is_you'] else 0.7),
      layer = "above"
      )
    all_dicts_of_images_to_add = (all_dicts_of_images_to_add
      + [this_ball_plot_dict]
      )

  shot_chart = go.Figure(
    data = go.Scatter(
      x = ball_at_goal_coordinates_df['ball_center_x'],
      y = ball_at_goal_coordinates_df['ball_center_y'],
      text = (
        'Player: ' + ball_at_goal_coordinates_df['player'] + 
        ', Kick #: ' + ball_at_goal_coordinates_df['kickno'].map(str) + 
        (('<br>Dist to Nearest Target: ' + 
           (ball_at_goal_coordinates_df['dist_to_nearest_target'].
             round(decimals = 3).map(str))) if add_dist_to_text else '')
        ),
      hoverinfo = plot_hoverinfo,
      hoverlabel = dict(
        bgcolor = np.where(ball_at_goal_coordinates_df['is_you'] == 1, 
          'darkgreen', 'grey'),
        bordercolor = 'black',
        font = dict(color = 'white')
        ),
      mode = 'markers'
      ),  
    layout = go.Layout(
      margin = dict(t = 0, r = 0, b = 0, l = 0),
      xaxis = dict(
        range = [figure_x_min, figure_x_max],
        showgrid = False,
        showticklabels = False
        ),
      yaxis = dict(
        # y-axis points are reversed, with 0 at top and 1 at bottom
        range = [figure_y_max, figure_y_min],
        showgrid = False,
        showticklabels = False      
        ),
      paper_bgcolor = 'rgba(0, 0, 0, 0)',
      plot_bgcolor = 'rgba(0, 0, 0, 0)',
      width = figure_height * figure_width_to_height_ratio, 
      height = figure_height,
      showlegend = False,
      images = all_dicts_of_images_to_add
      )
    )
  
  return(shot_chart)

goal_mouth_to_plot = dict(
  source = goal_mouth_image_file,
  xref = "x",
  yref = "y",
  xanchor = 'left',
  yanchor = 'bottom',
  x = 3.3/12.2, # Approximate x location of left goalpost, based on video
  y = 3.8/6.8, # Approximate y location of bottom of goalpost, based on video
  sizex = 5.5/12.2, # Approximate width of goal, based on video
  sizey = 1.9/6.8, # Approximate height of goal, based on video
  sizing = "stretch",
  opacity = 0.7,
  layer = "below"        
  )

# Get average target values to put red X's in goal on plot
left_target_x_median = ball_at_goal_coordinates['left_target_x'].median()
right_target_x_median = ball_at_goal_coordinates['right_target_x'].median()

target_y_median = np.median(
  np.concatenate((ball_at_goal_coordinates['right_target_y'], 
    ball_at_goal_coordinates['left_target_y']), axis = 0))

# Parameters for size of target
size_target_x = 0.04
size_target_y = size_target_x * (1920 /1080)

left_target_to_plot = dict(
  source = red_x_image_file,
  xref = "x",
  yref = "y",
  xanchor = 'left',
  yanchor = 'bottom',
  x = left_target_x_median,
  y = target_y_median + (size_target_y / 2),
  sizex = size_target_x,
  sizey = size_target_y,
  sizing = "stretch",
  opacity = 0.5,
  layer = "below"        
  )

right_target_to_plot = dict(
  source = red_x_image_file,
  xref = "x",
  yref = "y",
  xanchor = 'left',
  yanchor = 'bottom',
  x = right_target_x_median,
  y = target_y_median + (size_target_y / 2),
  sizex = size_target_x,
  sizey = size_target_y,
  sizing = "stretch",
  opacity = 0.5,
  layer = "below"        
  )

all_kicks_shot_chart = GetShotChartWithBallsAtGoalMouth(
  ball_at_goal_coordinates, [goal_mouth_to_plot, left_target_to_plot, right_target_to_plot],
  figure_x_min = 0.1, figure_x_max = 0.9,
  figure_y_min = 0, figure_y_max = 0.8
  )

all_kicks_shot_chart.show()

## Compare Your Kicks to Penalty Kicks from "Real" Matches
In this section, we dive deeper into penalty kick location data, looking at 1500+ penalty kicks from professional men's players in high-level club and country competitions across the last several years, courtesy of [Opta](https://www.optasports.com/). This includes World Cups, English Premier League, and more. These attempts are different from the kicks at our event since they occur in actual competition, where a goalkeeper can stop a penalty kick from going into the net. 

Below, we display an interactive shot chart (previously created and stored off using [Plotly](https://plot.ly/)) showing all these kicks by their ball locations at the goal mouth. Blue balls represent scores and the red 'X' balls represent misses/saves by the keeper. As above, use the mouse to hover over each ball for more info and zoom in/out of different sections, including some kicks that were **way** off the mark!,From this plot, we can get a decent idea of both shot density and success rate for penalty kicks at various points across the goal mouth.

In [0]:
#@title Interactive Shot Chart of "Real" Penalty Kicks from Professional Players
%%html
<iframe src="https://storage.googleapis.com/football-pier-colab/Penalty%20Kicks%20Shot%20Chart%20Interactive.html" width="1395" height="360"></iframe>

The chart below shows a heat map (also previously created and stored off using [Plotly](https://plot.ly/)) of the success rate of penalty kicks, using the same data from professional players as above. Success rates were smoothed across the goal mouth using a [generalized additive model (GAM)](https://www.statsmodels.org/dev/gam.html) over the ball location data. More blue represents higher success rates for kicks in that spot; more red represents areas with lower probability of scoring. Hoving over any point will show the estimated "score percentage" for a kick at that location.

In [0]:
#@title Interactive Heat Map of  Pros' Penalty Kick Success Rate by Location
%%html
<iframe src="https://storage.googleapis.com/football-pier-colab/Penalty%20Kicks%20Score%20Probability%20Heat%20Map%20Interactive.html" width="1395" height="360"></iframe>

For fun, we plot your kicks on an image of the heat map above to give an idea of how likely your kick would be to score in a "real" professional setting, *based on location alone*. Again, ***the goal mouth heat map image is placed approximately (not exactly) on the plot below, for illustration purposes.**

In [0]:
#@title Plot of Your Kicks on Heat Map of Pros' Penalty Kick Success Rates by Location
this_player_ball_at_goal_coordinates = ball_at_goal_coordinates[
  ball_at_goal_coordinates['is_you']]

heat_map_to_plot = goal_mouth_to_plot.copy()

# Change plot to take heat map as background instead of goal mouth
heat_map_to_plot['source'] = pk_score_prob_heat_map_image_file

player_kicks_on_heat_map = GetShotChartWithBallsAtGoalMouth(
  this_player_ball_at_goal_coordinates, [heat_map_to_plot],
  figure_x_min = 0.2, figure_x_max = 0.8,
  figure_y_min = 0.1, figure_y_max = 0.7,
  plot_hoverinfo = 'text', add_dist_to_text = False
  )

player_kicks_on_heat_map.show()

# [GOAL] Query _ball_-tracking data from the _goal_-facing camera

#### Try querying all of the ball-tracking data

In [0]:
goaldata = get_df_from_query('''
  SELECT * 
  FROM 
    `football-pier.goal.ball`
  LIMIT 10
  ''')
goaldata

,player,kickno,camera,ts_ball,target1_x,target1_y,target2_x,target2_y,ball_center_x,ball_center_y,ball_box_x1,ball_box_y1,ball_box_x2,ball_box_y2,impact
0,SR6BPH,3,goal,1.773850,0.683299,0.370039,0.320882,0.375376,0.218503,0.568847,0.204946,0.542755,0.232060,0.594939,0
1,SR6BPH,3,goal,1.798834,0.683299,0.370039,0.320882,0.375376,0.201672,0.548232,0.187738,0.524629,0.215605,0.571836,0
2,SR6BPH,3,goal,1.823818,0.683299,0.370039,0.320882,0.375376,0.186604,0.528777,0.175098,0.507754,0.198109,0.549800,0
3,SR6BPH,3,goal,1.748867,0.683299,0.370039,0.320882,0.375376,0.237934,0.596091,0.223773,0.570223,0.252096,0.621960,0
4,SR6BPH,3,goal,1.698899,0.683299,0.370039,0.320882,0.375376,0.280685,0.657656,0.263742,0.628779,0.297629,0.686533,0
5,SR6BPH,3,goal,1.898769,0.683299,0.370039,0.320882,0.375376,0.147637,0.485313,0.136690,0.465701,0.158583,0.504924,0
6,SR6BPH,3,goal,1.923753,0.683299,0.370039,0.320882,0.375376,0.135756,0.473692,0.124828,0.453600,0.146684,0.493785,0
7,SR6BPH,3,goal,1.773850,0.683299,0.370039,0.320882,0.375376,0.218503,0.568847,0.204946,0.542755,0.232060,0.594939,0
8,SR6BPH,3,goal,1.848802,0.683299,0.370039,0.320882,0.375376,0.171664,0.511443,0.159828,0.489902,0.183500,0.532984,0
9,SR6BPH,3,goal,1.798834,0.683299,0.370039,0.320882,0.375376,0.201672,0.548232,0.187738,0.524629,0.215605,0.571836,0


#### Query your data with your `PLAYER_ID`

In [0]:
print("My PLAYER_ID: {}".format(PLAYER_ID))

goaldata_me = get_df_from_query('''
  SELECT * 
  FROM 
    `football-pier.goal.ball`
  WHERE
    player = "%s"
  LIMIT 10
  ''' % (PLAYER_ID)
)

goaldata_me

My PLAYER_ID: 48XGTZ


,player,kickno,camera,ts_ball,target1_x,target1_y,target2_x,target2_y,ball_center_x,ball_center_y,ball_box_x1,ball_box_y1,ball_box_x2,ball_box_y2,impact
0,48XGTZ,3,goal,2.773165,0.684076,0.370679,0.320743,0.376703,0.668086,0.550802,0.653376,0.522820,0.682796,0.578785,0
1,48XGTZ,3,goal,2.798148,0.684076,0.370679,0.320743,0.376703,0.676547,0.501103,0.662551,0.474390,0.690543,0.527816,0
2,48XGTZ,3,goal,2.698214,0.684076,0.370679,0.320743,0.376703,0.625466,0.760585,0.605582,0.722629,0.645351,0.798541,0
3,48XGTZ,3,goal,2.723198,0.684076,0.370679,0.320743,0.376703,0.643177,0.676598,0.624875,0.642186,0.661479,0.711009,0
4,48XGTZ,1,goal,3.472762,0.320743,0.376807,0.682626,0.370507,0.664804,0.454253,0.652003,0.432971,0.677604,0.475535,0
5,48XGTZ,3,goal,3.247850,0.684076,0.370679,0.320743,0.376703,0.719814,0.223849,0.712048,0.209922,0.727580,0.237777,1
6,48XGTZ,3,goal,2.848115,0.684076,0.370679,0.320743,0.376703,0.693420,0.428430,0.680976,0.406230,0.705865,0.450630,0
7,48XGTZ,2,goal,2.748250,0.682799,0.370827,0.320803,0.377400,0.657122,0.575109,0.641711,0.546936,0.672533,0.603283,0
8,48XGTZ,1,goal,3.497746,0.320743,0.376807,0.682626,0.370507,0.663298,0.427595,0.651802,0.407027,0.674795,0.448163,1
9,48XGTZ,2,goal,2.673298,0.682799,0.370827,0.320803,0.377400,0.630873,0.769646,0.610239,0.730170,0.651506,0.809121,0


#### What do the columns of `football-pier.goal.ball` represent?

In [0]:
get_df_from_query('''
SELECT
 * EXCEPT(table_catalog, table_schema, table_name, field_path)
FROM
 `football-pier`.goal.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
WHERE
 table_name="ball"
  ''')

,column_name,data_type,description
0,player,STRING,"Player ID, anonymized"
1,kickno,INT64,Kick number
2,camera,STRING,Camera angle
3,ts_ball,FLOAT64,Timestamp in seconds
4,target1_x,FLOAT64,"X co-ord of center of target1, normalized betw..."
5,target1_y,FLOAT64,"Y co-ord of center of target1, normalized betw..."
6,target2_x,FLOAT64,"X co-ord of center of target2, normalized betw..."
7,target2_y,FLOAT64,"Y co-ord of center of target2, normalized betw..."
8,ball_center_x,FLOAT64,"X co-ord of center of ball_center, normalized ..."
9,ball_center_y,FLOAT64,"Y co-ord of center of ball_center, normalized ..."


<hr>

# [SIDE] Query _ball_-tracking data from the _side_-facing camera

#### Try querying all of the ball-tracking data

In [0]:
sidedata = get_df_from_query('''
  SELECT * 
  FROM 
    `football-pier.side.ball`
  LIMIT 10
  ''')
sidedata

,player,kickno,camera,ts_ball,ball_center_x,ball_center_y,ball_box_x1,ball_box_y1,ball_box_x2,ball_box_y2
0,6X8TCF,3,side,0.199864,NaN,NaN,NaN,NaN,NaN,NaN
1,6X8TCF,3,side,4.796745,NaN,NaN,NaN,NaN,NaN,NaN
2,6X8TCF,3,side,4.197152,NaN,NaN,NaN,NaN,NaN,NaN
3,6X8TCF,3,side,3.497627,0.103105,0.246439,0.082284,0.212153,0.123926,0.280726
4,6X8TCF,3,side,3.714147,0.563690,0.172687,0.542019,0.134000,0.585360,0.211374
5,6X8TCF,3,side,4.380361,NaN,NaN,NaN,NaN,NaN,NaN
6,6X8TCF,3,side,3.181175,NaN,NaN,NaN,NaN,NaN,NaN
7,6X8TCF,3,side,4.979954,NaN,NaN,NaN,NaN,NaN,NaN
8,6X8TCF,3,side,3.064587,NaN,NaN,NaN,NaN,NaN,NaN
9,6X8TCF,3,side,3.664180,0.449643,0.182684,0.428170,0.143272,0.471116,0.222097


#### Query your data with your `PLAYER_ID`

In [0]:
print("My PLAYER_ID: {}".format(PLAYER_ID))

sidedata_me = get_df_from_query('''
  SELECT * 
  FROM 
    `football-pier.side.ball`
  WHERE
    player = "%s"
  LIMIT 10
  ''' % (PLAYER_ID)
)

sidedata_me

My PLAYER_ID: 48XGTZ


,player,kickno,camera,ts_ball,ball_center_x,ball_center_y,ball_box_x1,ball_box_y1,ball_box_x2,ball_box_y2
0,48XGTZ,2,side,2.764843,0.114263,0.106813,0.085475,0.058914,0.143051,0.154713
1,48XGTZ,3,side,2.798258,0.013573,0.127619,0.000000,0.086822,0.027146,0.168417
2,48XGTZ,1,side,3.414383,0.013380,0.155926,0.001000,0.110545,0.025760,0.201307
3,48XGTZ,1,side,3.431039,0.051494,0.115791,0.022051,0.066703,0.080936,0.164879
4,48XGTZ,2,side,2.798154,0.236228,0.040567,0.204288,0.009701,0.268169,0.071434
5,48XGTZ,1,side,3.447694,0.114072,0.075765,0.085131,0.027998,0.143014,0.123532
6,48XGTZ,2,side,2.748187,0.055212,0.144488,0.026093,0.096598,0.084331,0.192378
7,48XGTZ,3,side,2.814914,0.045259,0.080718,0.015227,0.032844,0.075290,0.128592
8,48XGTZ,1,side,3.481005,0.244062,0.028898,0.222819,0.000000,0.265305,0.057797
9,48XGTZ,2,side,2.781498,0.169125,0.066955,0.138247,0.021715,0.200004,0.112195


#### What do the columns of `football-pier.side.ball` represent?

In [0]:
get_df_from_query('''
SELECT
 * EXCEPT(table_catalog, table_schema, table_name, field_path)
FROM
 `football-pier`.side.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
WHERE
 table_name="ball"
  '''
  )

,column_name,data_type,description
0,player,STRING,"Player ID, anonymized"
1,kickno,INT64,Kick number
2,camera,STRING,Camera angle
3,ts_ball,FLOAT64,Timestamp in seconds
4,ball_center_x,FLOAT64,"X co-ord of center of ball_center, normalized ..."
5,ball_center_y,FLOAT64,"Y co-ord of center of ball_center, normalized ..."
6,ball_box_x1,FLOAT64,X co-ord of upper-left corner of bounding box ...
7,ball_box_y1,FLOAT64,Y co-ord of upper-left corner of bounding box ...
8,ball_box_x2,FLOAT64,X co-ord of bottom-right corner of bounding bo...
9,ball_box_y2,FLOAT64,Y co-ord of bottom-right corner of bounding bo...


<hr>

# [KICKER] Query _pose_-detection data from the _kicker_-facing camera

#### Try querying all of the pose-detection data

In [0]:
posedata = get_df_from_query('''
  SELECT * 
  FROM 
    `football-pier.kicker.pose`
  LIMIT 10
  ''')
posedata

,player,kickno,camera,ts_pose,nose,nose_conf,nose_x,nose_y,left_eye,left_eye_conf,left_eye_x,left_eye_y,right_eye,right_eye_conf,right_eye_x,right_eye_y,left_ear,left_ear_conf,left_ear_x,left_ear_y,right_ear,right_ear_conf,right_ear_x,right_ear_y,left_shoulder,left_shoulder_conf,left_shoulder_x,left_shoulder_y,right_shoulder,right_shoulder_conf,right_shoulder_x,right_shoulder_y,left_elbow,left_elbow_conf,left_elbow_x,left_elbow_y,right_elbow,right_elbow_conf,right_elbow_x,right_elbow_y,left_wrist,left_wrist_conf,left_wrist_x,left_wrist_y,right_wrist,right_wrist_conf,right_wrist_x,right_wrist_y,left_hip,left_hip_conf,left_hip_x,left_hip_y,right_hip,right_hip_conf,right_hip_x,right_hip_y,left_knee,left_knee_conf,left_knee_x,left_knee_y,right_knee,right_knee_conf,right_knee_x,right_knee_y,left_ankle,left_ankle_conf,left_ankle_x,left_ankle_y,right_ankle,right_ankle_conf,right_ankle_x,right_ankle_y
0,2THDGD,4,kicker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2THDGD,4,kicker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,XQZBCF,4,kicker,1.4,1.0,0.888288,0.126541,0.193641,1.0,0.054275,0.123736,0.183673,1.0,0.958500,0.119529,0.186165,NaN,NaN,NaN,NaN,1.0,0.920206,0.102701,0.203610,1.0,0.203611,0.115322,0.260927,1.0,0.572648,0.111115,0.285847,1.0,0.059709,0.151783,0.313260,1.0,0.580069,0.154588,0.318244,1.0,0.033158,0.214888,0.330704,1.0,0.795209,0.216291,0.330704,1.0,0.636951,0.170013,0.415434,1.0,0.682044,0.147576,0.425402,1.0,0.845943,0.199463,0.520100,1.0,0.852128,0.148978,0.545020,1.0,0.872276,0.209279,0.639718,1.0,0.774357,0.099896,0.667131
3,92QPJL,1,kicker,0.8,1.0,1.007249,0.056437,0.190821,1.0,1.011656,0.062077,0.184808,1.0,1.011198,0.051926,0.184808,1.0,0.964995,0.069972,0.19483,1.0,0.658504,0.047414,0.192825,1.0,0.873973,0.083506,0.240928,1.0,0.920844,0.036136,0.250950,1.0,0.920637,0.102680,0.289031,1.0,0.890725,0.031624,0.323104,1.0,0.856896,0.097040,0.339138,1.0,0.883680,0.047414,0.377219,1.0,0.636111,0.084634,0.363189,1.0,0.641741,0.056437,0.369202,1.0,0.897472,0.089145,0.451377,1.0,0.890421,0.057565,0.455386,1.0,0.889224,0.089145,0.531548,1.0,0.815055,0.053054,0.541570
4,98TCPB,2,kicker,4.3,1.0,0.955831,0.041715,0.192622,1.0,0.017854,0.040063,0.177940,1.0,0.974811,0.036758,0.177940,NaN,NaN,NaN,NaN,1.0,0.946879,0.015277,0.177940,1.0,0.446630,0.013625,0.251348,1.0,0.447724,0.002059,0.257220,1.0,0.304908,0.018582,0.368800,1.0,0.242052,0.005363,0.377608,1.0,0.322535,0.040063,0.424589,1.0,0.649408,0.025192,0.459825,1.0,0.311798,0.021887,0.445143,1.0,0.411522,0.005363,0.453952,1.0,0.741274,0.059891,0.544977,1.0,0.088417,-0.001246,0.577277,1.0,0.804268,0.071457,0.682984,1.0,0.086644,-0.002898,0.732901
5,8TVGVB,2,kicker,4.3,1.0,0.893706,0.404468,0.173500,1.0,0.022761,0.403109,0.166254,1.0,0.886006,0.401749,0.166254,NaN,NaN,NaN,NaN,1.0,0.826656,0.390875,0.171085,1.0,0.365430,0.377282,0.219396,1.0,0.701853,0.378641,0.224227,1.0,0.426754,0.401749,0.299109,1.0,0.735420,0.385438,0.311187,1.0,0.717064,0.435732,0.318433,1.0,0.767548,0.407186,0.388485,1.0,0.478236,0.380000,0.378822,1.0,0.494551,0.394953,0.381238,1.0,0.842272,0.392234,0.485107,1.0,0.849307,0.427576,0.480276,1.0,0.878771,0.385438,0.593807,1.0,0.827878,0.439810,0.591391
6,6D8LLH,1,kicker,5.3,1.0,0.856688,0.270743,0.183577,1.0,0.025499,0.267660,0.172620,1.0,0.909942,0.264577,0.175359,NaN,NaN,NaN,NaN,1.0,0.908947,0.249161,0.189056,1.0,0.427333,0.241454,0.243844,1.0,0.841702,0.236829,0.252062,1.0,0.062576,0.236829,0.342461,1.0,0.716843,0.224497,0.345201,1.0,0.031811,0.241454,0.413685,1.0,0.679242,0.226039,0.424643,1.0,0

#### Query your data with your `PLAYER_ID`:

In [0]:
print("My PLAYER_ID: {}".format(PLAYER_ID))

posedata_me = get_df_from_query('''
  SELECT * 
  FROM 
    `football-pier.kicker.pose`
  WHERE
    player = "%s"
  LIMIT 10
  ''' % (PLAYER_ID)
)

posedata_me

My PLAYER_ID: 48XGTZ


,player,kickno,camera,ts_pose,nose,nose_conf,nose_x,nose_y,left_eye,left_eye_conf,left_eye_x,left_eye_y,right_eye,right_eye_conf,right_eye_x,right_eye_y,left_ear,left_ear_conf,left_ear_x,left_ear_y,right_ear,right_ear_conf,right_ear_x,right_ear_y,left_shoulder,left_shoulder_conf,left_shoulder_x,left_shoulder_y,right_shoulder,right_shoulder_conf,right_shoulder_x,right_shoulder_y,left_elbow,left_elbow_conf,left_elbow_x,left_elbow_y,right_elbow,right_elbow_conf,right_elbow_x,right_elbow_y,left_wrist,left_wrist_conf,left_wrist_x,left_wrist_y,right_wrist,right_wrist_conf,right_wrist_x,right_wrist_y,left_hip,left_hip_conf,left_hip_x,left_hip_y,right_hip,right_hip_conf,right_hip_x,right_hip_y,left_knee,left_knee_conf,left_knee_x,left_knee_y,right_knee,right_knee_conf,right_knee_x,right_knee_y,left_ankle,left_ankle_conf,left_ankle_x,left_ankle_y,right_ankle,right_ankle_conf,right_ankle_x,right_ankle_y
0,48XGTZ,1,kicker,3.5,1.0,0.973590,0.233616,0.386825,1.0,0.675426,0.235584,0.372839,1.0,0.957103,0.225746,0.376336,None,None,None,None,1.0,0.903445,0.206070,0.386825,1,0.685796,0.245421,0.418293,1,0.743430,0.186395,0.453258,1,0.738826,0.282805,0.439272,1.0,0.812199,0.143109,0.505704,1,0.697967,0.320188,0.484726,1,0.885249,0.113596,0.575633,1,0.262671,0.269032,0.565144,1.0,0.408665,0.245421,0.579130,1.0,0.428832,0.269032,0.694512,1.0,0.197854,0.322156,0.586123,1.0,0.806033,0.229681,0.813392,1.0,0.520319,0.389052,0.582626
1,48XGTZ,1,kicker,3.6,1.0,0.947468,0.258568,0.376256,1.0,0.571810,0.258568,0.364680,1.0,0.961253,0.252054,0.368538,None,None,None,None,1.0,0.883483,0.230340,0.380114,1,0.747633,0.273768,0.410984,1,0.742601,0.212969,0.449570,1,0.661915,0.308510,0.453429,1.0,0.810957,0.173884,0.511309,1,0.689969,0.332395,0.503591,1,0.895780,0.143485,0.576906,1,0.233043,0.295482,0.561471,1.0,0.402592,0.275939,0.576906,1.0,0.487141,0.288967,0.688807,1.0,0.158170,0.343252,0.565330,1.0,0.821145,0.239026,0.800708,1.0,0.462649,0.423593,0.603917
2,48XGTZ,1,kicker,2.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,NaN,NaN,1,0.416430,0.005802,0.427139,1,0.011059,0.005054,0.425809,1,0.643626,0.021512,0.501586,NaN,NaN,NaN,NaN,1,0.649358,0.026749,0.578692,1,0.028491,0.026749,0.580021,1,0.052438,0.002061,0.577362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,48XGTZ,1,kicker,3.8,1.0,0.901933,0.320680,0.398712,1.0,0.362090,0.318508,0.387136,1.0,0.939488,0.311995,0.390995,None,None,None,None,1.0,0.888577,0.292453,0.406429,1,0.510885,0.325022,0.441155,1,0.699250,0.288111,0.464306,1,0.295265,0.357591,0.452730,1.0,0.756674,0.259884,0.537616,1,0.539210,0.353249,0.468164,1,0.822348,0.249028,0.607069,1,0.258566,0.340221,0.607069,1.0,0.343638,0.325022,0.614786,1.0,0.555384,0.309823,0.730540,1.0,0.728952,0.387989,0.680380,1.0,0.743488,0.246857,0.799992,1.0,0.812896,0.396674,0.807709
4,48XGTZ,1,kicker,4.1,1.0,0.917743,0.368924,0.397383,1.0,0.098225,0.367597,0.385589,1.0,0.959191,0.362287,0.387948,None,None,None,None,1.0,0.929998,0.343705,0.397383,1,0.431339,0.354324,0.439839,1,0.689499,0.333087,0.463425,1,0.200359,0.374233,0.470501,1.0,0.614946,0.378215,0.517675,1,0.315544,0.380869,0.463425,1,0.679245,0.425997,0.510599,1,0.199401,0.338396,0.604945,1.0,0.471467,0.347687,0.609663,1.0,0.699955,0.333087,0.727596,1.0,0.798437,0.406088,0.673347,1.0,0.776984,0.273359,0.791280,1.0,0.771977,0.398124,0.803073
5,48XGTZ,1,kicker,4.5,1.0,0.895005,0.333711,0.374721,1.0,0.027697,0.332359,0.365111,1.0,0.947277,0.328304,0.365111,None,None,None,None,1.0,0.916815,0.309377,0.377124,1,0.829581,0.282339,0.427575,1,0.655352,0.321544,0.429977,1,0.048041,0.316137,0.475623,1.0,0.751849,0.358046,0.497245,1,0.243941,0.343175,0.441989,1,0.264484,0.348582,0.547696,1,0.552170,0.290450,0.590939,1.0,0.474713,0.324248,0.590939,1.0,0.811425,0.306673,0.689439,1.0,0.819929,0.363453,0.684634,1.0,0.797952,0.259356,0.785536,1.0,0.802166,0.391844,0.799951
6,48XGTZ,1,kicker,5.1,1.0,0.977277,0.283433,0.343975,1.0,0.298998,0.282042,0.334086,1.0

#### What do the columns of `football-pier.kicker.pose` represent?

In [0]:
get_df_from_query('''
SELECT
 * EXCEPT(table_catalog, table_schema, table_name, field_path)
FROM
 `football-pier`.kicker.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
WHERE
 table_name="pose"
  ''')

,column_name,data_type,description
0,player,STRING,"Player ID, anonymized"
1,kickno,INT64,Kick number
2,camera,STRING,Camera angle
3,ts_pose,FLOAT64,Timestamp in seconds
4,nose,INT64,1 if landmark detected else 0
...,...,...,...
67,left_ankle_y,FLOAT64,"Y co-ord of center of left_ankle, normalized b..."
68,right_ankle,INT64,1 if landmark detected else 0
69,right_ankle_conf,FLOAT64,"Confidence of pose prediction F, between 0 (no..."
70,right_ankle_x,FLOAT64,"X co-ord of center of right_ankle, normalized ..."
